# Python Example Interacting with AWS Cognito and API Gateway

This purpose of this notebook is to provide some working examples of an Python client signing up, confirming signup and login operations using Boto3 and the Cognito client.

This notebook goes along with a blog post that can be found TBD

In [49]:
from dotenv import load_dotenv
import boto3
import boto3
import botocore.exceptions
import hmac
import hashlib
import base64
import json
import os
from jose import jwt
import requests
from pprint import pprint

Create a file called ```.env``` in the same directory as this Jupyter notebook.

All sensitive data is kept in a ```.env``` file that looks like the following:

```
# cognito pool creds - NEVER COMMIT THIS FILE
USER_POOL_ID=us-east-1_XE ... tK
CLIENT_ID=3v0 ... 8hlh5
CLIENT_SECRET=19f3t ... er2f5ojg19
EMAIL=you@somewhere.com
PASSWORD=yourfavpassword
AWS_PROFILE=profile name to access AWS
AWS_API_GATEWAY_URL=https://abcdefghij.execute-api.us-east-1.amazonaws.com/stage/resource
```

In [56]:
load_dotenv(override=True)

True

In [57]:
email = os.getenv('EMAIL')
password = os.getenv('PASSWORD')
aws_profile_name=os.getenv('AWS_PROFILE')

### Decode and Verify Cognito Token

To decode and verify that the token is valid, use the Jose JWT Python library.

In addition you need the JWKS keys which can be found at:

```
https://cognito-idp.{REGION}.amazonaws.com/{USER POOL ID}/.well-known/jwks.json
```



In [52]:
def decode_verify_token(token):
    """
    {
       "keys":[
          {
             "alg":"RS256",
             "e":"AQAB",
             "kid":"hmlIi0ie4TJZKVbGCbWV20w1qqsNPyjoFhau1MfS+i4=",
             "kty":"RSA",
             "n":"2IASx_zrg9cIuj1I4LblmQLGHfnNWSManTbvMXjp9_LI6X5fyv2pjLHsOAjga_vX3776JkSDx5Fv6IoSWWlrwytSlb57y-0GCvox0mK_KEczFzBVJUOhJiHmZKhTcMwf2NSU4yJ6srzuoKnSuq1q3kLzKgRSOQWIUycufvmhqaVu92Jr0vCklTKu2qgD7j1WmRYN9m0dylYpaI2Vybol8zaCcP3ft_eW-S_W9e0IjKOmH-KZ6_NHJCtfo92KgrBYC7W0kcBsXJymmarXVuvPwxY33-Mz7tTnVJHuKJsNhQWe8615G6VyOxYHRir1tn0p6Bl6Y3jnw8pitJGINjL2pw",
             "use":"sig"
          },
          {
             "alg":"RS256",
             "e":"AQAB",
             "kid":"osr5Yj3xOSkxzHx4P9uDT7kYq0T92OUruEkimanAHec=",
             "kty":"RSA",
             "n":"qwAyACmQZ_mece3vKnte6p3BTeOvmq47F1JjNtfV7pbQyJxvFycDt5l0CDGqBoFwm3bIU-nweXFJ_dCVFiJ3M_nGm4OcBqKp16tzhQy6Twc-x1tcabmfCT2Vb1GCOnlW3dJdRY3SuHq-wHboHyPg18e4y4qZ8bC0qBRMdc5MWdUC2EuHYds7s9FZES7ECMfNpFhYq5LLG_DqxPArgl1d86M4xviJVr_0CBZJZ3sMWosxZOWpJpEEkjbrPTYHtYIgOM3jy5kknj1G8I_tz_qzzkYk1NCXODgGiLtnAqOAh1v8E7t5OM5SSTlHI42CxJY5ClJayCBpyLCszEy312Taow",
             "use":"sig"
          }
       ]
    }
    """
    # build the URL where the public keys are
    jwks_url = 'https://cognito-idp.{}.amazonaws.com/{}/.well-known/jwks.json'.format(
                        'us-east-1',
                        os.getenv('USER_POOL_ID'))
    # get the keys
    jwks = requests.get(jwks_url).json()
    decoded_token = jwt.decode(token, key=jwks, options={'verify_aud':False, 'verify_signature':False})
#     pprint(decoded_token)
    return decoded_token


### Secret Hash

When creating a python client, you need to create the client secret which is then used in the secret hash.  The calculation of that is described in this document:

```
https://docs.aws.amazon.com/cognito/latest/developerguide/signing-up-users-in-your-app.html
```

and an implementation is below.

In [53]:
def get_secret_hash(username, cognito_client_id, cognito_secret):
    msg = username + cognito_client_id
    dig = hmac.new(str(cognito_secret).encode('utf-8'), 
        msg = str(msg).encode('utf-8'), digestmod=hashlib.sha256).digest()
    d2 = base64.b64encode(dig).decode()
    return d2

### Coginto Helper Functions

Below are some helper functions used to signup, confirm, and login a user based on an email address and password.


In [71]:
def cognito_signup(email, password, name, favorite_band):
    """
    GOTCHA:  Since we are using the email address as the username, according to this S.O.
        https://stackoverflow.com/questions/54430978/unable-to-verify-secret-hash-for-client-at-refresh-token-auth
        
    when you have an "@" in the username you cannot use the email address for REFRESH_TOKEN_AUTH.  Cognito
    generates a UUID-style username for those users.  When creating the secret hash for the REFRESH_TOKEN_AUTH
    you have to use the Cognito generated username.
    
    """
    session = boto3.Session(profile_name=aws_profile_name)
    client = session.client('cognito-idp')
    
    try:
        resp = client.sign_up(
            ClientId = os.getenv('CLIENT_ID'),
            SecretHash=get_secret_hash(email, os.getenv('CLIENT_ID'), os.getenv('CLIENT_SECRET')),
            Username=email,
            Password=password,
            UserAttributes=[
                {
                    "Name": "custom:favorite_band",
                    "Value": favorite_band
                },
                {
                    "Name":"name",
                    "Value":name
                }
            ]
            
        )
        
    except client.exceptions.UsernameExistsException as e:
        return {
               "success": False, 
               "message": "This username already exists", 
               "data": None}
    except client.exceptions.InvalidPasswordException as e:
        
        return {
               "success": False, 
               "message": "Password should have Caps,\
                          Special chars, Numbers", 
               "data": None}
    except client.exceptions.UserLambdaValidationException as e:
        return {
               "success": False, 
               "message": "Email already exists", 
               "data": None}
    
    except Exception as e:
        return {
                "success": False, 
                "message": str(e), 
               "data": None}
    
    return {
            "success": True, 
            "message": "Please confirm your signup, \
                        check Email for validation code", 
            "data": None}

def cognito_confirm_signup(email, code):
    session = boto3.Session(profile_name=aws_profile_name)
    client = session.client('cognito-idp')
    
    try:
        resp = client.confirm_sign_up(
            ClientId = os.getenv('CLIENT_ID'),
            SecretHash=get_secret_hash(email, os.getenv('CLIENT_ID'), os.getenv('CLIENT_SECRET')),
            Username=email,
            ConfirmationCode=code,
            ForceAliasCreation=False
            
        )

        return {"success": True, "message": f"Signup confirmed for user: {email}", "response": resp}

    except client.exceptions.UserNotFoundException:
        return {"success": False, "message": "Username doesnt exists"}
    except client.exceptions.CodeMismatchException:
        return {"success": False, "message": "Invalid Verification code"}
    except client.exceptions.NotAuthorizedException:
        return {"success": False, "message": "User is already confirmed"}
    except Exception as e:
        return {"success": False, "message": f"Unknown error {e.__str__()} "}
    

def cognito_login(email, password):
    session = boto3.Session(profile_name=aws_profile_name)
    client = session.client('cognito-idp')

    try:
        resp=client.initiate_auth(
                 ClientId=os.getenv('CLIENT_ID'),
                 AuthFlow='USER_PASSWORD_AUTH',
                 AuthParameters={
                     'USERNAME': email,
                     'SECRET_HASH': get_secret_hash(email, os.getenv('CLIENT_ID'), os.getenv('CLIENT_SECRET')),
                     'PASSWORD': password,
                  })

        return resp
    
    except client.exceptions.NotAuthorizedException:
        return None, "The username or password is incorrect"
    except client.exceptions.UserNotConfirmedException:
        return None, "User is not confirmed"
    except Exception as e:
        return None, e.__str__()


def cognito_refresh_token(original_token, refresh_token):
    """
    GOTCHA:  Since we are using the email address as the username, according to this S.O.
        https://stackoverflow.com/questions/54430978/unable-to-verify-secret-hash-for-client-at-refresh-token-auth
        
    when you have an "@" in the username you cannot use the email address for REFRESH_TOKEN_AUTH.  Cognito
    generates a UUID-style username for those users.  When creating the secret hash for the REFRESH_TOKEN_AUTH
    you have to use the Cognito generated username.
    
    """


    session = boto3.Session(profile_name=aws_profile_name)
    client = session.client('cognito-idp')
    

    try:
        decoded_token = jwt.get_unverified_claims(original_token)
        print(decoded_token)
        cognito_username = decoded_token['cognito:username']
        print(cognito_username)
    
        resp=client.initiate_auth(
                 ClientId=os.getenv('CLIENT_ID'),
                 AuthFlow='REFRESH_TOKEN_AUTH',
                 AuthParameters={
                     'REFRESH_TOKEN': refresh_token,
                     'SECRET_HASH': get_secret_hash(cognito_username, os.getenv('CLIENT_ID'), os.getenv('CLIENT_SECRET'))
                  }
            )

        return resp
    
    except client.exceptions.NotAuthorizedException as e:
        return None, "Not Authorized: " + e.__str__()
    except client.exceptions.UserNotConfirmedException:
        return None, "User is not confirmed"
    except Exception as e:
        return None, e.__str__()


    

In [58]:
cognito_signup(email, password, "Aunt Bea", "Lawerence Welk")

{'success': True,
 'message': 'Please confirm your signup,                         check Email for validation code',
 'data': None}

After the call to signup, you should receive an email at the address you provided above with a confirmation code.  Type that code below to confirm the signup.

In [59]:
cognito_confirm_signup(email, '013037')

{'success': True,
 'message': 'Signup confirmed for user: aunt.bea@contbay.com',
 'response': {'ResponseMetadata': {'RequestId': '53a67b71-d3c8-4c47-8ca0-5b269db168bd',
   'HTTPStatusCode': 200,
   'HTTPHeaders': {'date': 'Thu, 12 Mar 2020 01:38:40 GMT',
    'content-type': 'application/x-amz-json-1.1',
    'content-length': '2',
    'connection': 'keep-alive',
    'x-amzn-requestid': '53a67b71-d3c8-4c47-8ca0-5b269db168bd'},
   'RetryAttempts': 0}}}

## Login Example

<b>NOTE</b>

To allow for username/password login you have to enable username password login.

User Pool -> General Settings -> App clients -> Show Details -> Enable username password based authentication (ALLOW_USER_PASSWORD_AUTH)

In [60]:
login_response = cognito_login(email, password)

In [61]:
login_response

{'ChallengeParameters': {},
 'AuthenticationResult': {'AccessToken': 'eyJraWQiOiJvc3I1WWozeE9Ta3h6SHg0UDl1RFQ3a1lxMFQ5Mk9VcnVFa2ltYW5BSGVjPSIsImFsZyI6IlJTMjU2In0.eyJzdWIiOiJlZTBiZWRjZi0zYWQ4LTQ0MmEtOGFhYS0wMTU1NzBkYzk1N2UiLCJldmVudF9pZCI6IjMyMjk3N2M5LWUxYTMtNGZmOS1hYjYxLTk3ZWE4MGRkOTUwNCIsInRva2VuX3VzZSI6ImFjY2VzcyIsInNjb3BlIjoiYXdzLmNvZ25pdG8uc2lnbmluLnVzZXIuYWRtaW4iLCJhdXRoX3RpbWUiOjE1ODM5NzcxNDMsImlzcyI6Imh0dHBzOlwvXC9jb2duaXRvLWlkcC51cy1lYXN0LTEuYW1hem9uYXdzLmNvbVwvdXMtZWFzdC0xX1hFdTRjVnZ0SyIsImV4cCI6MTU4Mzk4MDc0MywiaWF0IjoxNTgzOTc3MTQzLCJqdGkiOiJiNTBjYTcwYy03NTczLTQzNzQtOTgyOC0yYjdlZTFmYzc2N2UiLCJjbGllbnRfaWQiOiIzdjBlMzc3dDF2dDY5amNycWVvNTg4aGxoNSIsInVzZXJuYW1lIjoiZWUwYmVkY2YtM2FkOC00NDJhLThhYWEtMDE1NTcwZGM5NTdlIn0.JXKMBVpr3RUaRWwTx2OzxOa0oygfj7mEX7HRDnjwQAZM0sz1xupZSQbRlzAjKyZcxRBohecHYE6yXzPCeSxwf_yMMPLoKxTV0weSsJRCk07gP-B9PMpoIzesDv-0utEIkqGbalV7QlIKKKGg9bgPUihu6OP7ppg_Xv5HA1h_pTIwOtXKDRY3VEDU1cEloBjpQHg0Rwyjbnp2n_x5V2IU-2BgYA1qp9gkydyJPHHiiXsSL0oQtvqNxwIPgkXznjHRe_RaBgZJGmnEmy

Get the IdToken that will be used when making AWS API Gateway calls.

In [62]:
IdToken = login_response['AuthenticationResult']['IdToken']

In [63]:
IdToken

'eyJraWQiOiJobWxJaTBpZTRUSlpLVmJHQ2JXVjIwdzFxcXNOUHlqb0ZoYXUxTWZTK2k0PSIsImFsZyI6IlJTMjU2In0.eyJzdWIiOiJlZTBiZWRjZi0zYWQ4LTQ0MmEtOGFhYS0wMTU1NzBkYzk1N2UiLCJlbWFpbF92ZXJpZmllZCI6dHJ1ZSwiaXNzIjoiaHR0cHM6XC9cL2NvZ25pdG8taWRwLnVzLWVhc3QtMS5hbWF6b25hd3MuY29tXC91cy1lYXN0LTFfWEV1NGNWdnRLIiwiY29nbml0bzp1c2VybmFtZSI6ImVlMGJlZGNmLTNhZDgtNDQyYS04YWFhLTAxNTU3MGRjOTU3ZSIsImF1ZCI6IjN2MGUzNzd0MXZ0NjlqY3JxZW81ODhobGg1IiwiZXZlbnRfaWQiOiIzMjI5NzdjOS1lMWEzLTRmZjktYWI2MS05N2VhODBkZDk1MDQiLCJ0b2tlbl91c2UiOiJpZCIsImF1dGhfdGltZSI6MTU4Mzk3NzE0MywibmFtZSI6IkF1bnQgQmVhIiwiZXhwIjoxNTgzOTgwNzQzLCJpYXQiOjE1ODM5NzcxNDMsImN1c3RvbTpmYXZvcml0ZV9iYW5kIjoiTGF3ZXJlbmNlIFdlbGsiLCJlbWFpbCI6ImF1bnQuYmVhQGNvbnRiYXkuY29tIn0.SHmSAdUCEReoWMLdc7n_ARF9xvhy9BrL1S63-BNnEQJBht2T6Avd0uO6y1oHf5k0vPv_XsSlWg4cHO_O0dJmaKpeWIfiSSmiPg1j24q7NFczxCAngUS9pHbO10s-IFqb56TSvlaI31FRfF3GZLyILLxq5vsnWehTjl2gi87LeC7XQeh36xZ_IOf3vAvkB1D_xuz0ZHd9zY9QYWuFYU0JnyATH3yyASdXD8Rc3Uy5HmiV_Zr9G06hDcfeG5qsrf58wB-y-lrwtOvbMM3jmPYXSSQc4D0RB8xCisvLvv-PyPAFaq4rJCw

Using the Jose JWT library, inspect the claims to see what data is available in the token.  Once this token is passed to a Lambda function, the claims can be used to associate data or actions with a user.  

Notice below that since we setup the Cognito User Pool to use email for the Username, Cognito generated a random username for us.  Further processing in the Lambda can use either the cognitor:username or email as the key to perform operations on this users behalf.


In [64]:
jwt.get_unverified_claims(IdToken)

{'sub': 'ee0bedcf-3ad8-442a-8aaa-015570dc957e',
 'email_verified': True,
 'iss': 'https://cognito-idp.us-east-1.amazonaws.com/us-east-1_XEu4cVvtK',
 'cognito:username': 'ee0bedcf-3ad8-442a-8aaa-015570dc957e',
 'aud': '3v0e377t1vt69jcrqeo588hlh5',
 'event_id': '322977c9-e1a3-4ff9-ab61-97ea80dd9504',
 'token_use': 'id',
 'auth_time': 1583977143,
 'name': 'Aunt Bea',
 'exp': 1583980743,
 'iat': 1583977143,
 'custom:favorite_band': 'Lawerence Welk',
 'email': 'aunt.bea@contbay.com'}

In [65]:
# write token to file to be used later
# can be used to verify the tokens do expire, or disabled users can no longer use the tokens.
f = open("./token.txt", "w")
f.write(IdToken)
f.close()

In [66]:
decode_verify_token(IdToken)

{'sub': 'ee0bedcf-3ad8-442a-8aaa-015570dc957e',
 'email_verified': True,
 'iss': 'https://cognito-idp.us-east-1.amazonaws.com/us-east-1_XEu4cVvtK',
 'cognito:username': 'ee0bedcf-3ad8-442a-8aaa-015570dc957e',
 'aud': '3v0e377t1vt69jcrqeo588hlh5',
 'event_id': '322977c9-e1a3-4ff9-ab61-97ea80dd9504',
 'token_use': 'id',
 'auth_time': 1583977143,
 'name': 'Aunt Bea',
 'exp': 1583980743,
 'iat': 1583977143,
 'custom:favorite_band': 'Lawerence Welk',
 'email': 'aunt.bea@contbay.com'}

### Refresh Token


In [67]:
refresh_token = login_response['AuthenticationResult']['RefreshToken']

In [68]:
refresh_token

'eyJjdHkiOiJKV1QiLCJlbmMiOiJBMjU2R0NNIiwiYWxnIjoiUlNBLU9BRVAifQ.GPlhVJgRzmkNd3TY9ALl5nka98XnTnEzbPnDvVo95L0t4obnH9dse1ADVxCoLLL2H9IplzZkNZ3D1dAzRknzXcGsBYbPGKG4lfnJxuK8fHOrgyXnRIOZy3PgF5-sbDB4GpHTwkNJfHseWHNcrifqg14pjacTvb5p5EW8DHAfvOXWTAHavJk2H9z_uSlLY6udvhl2csTkkWbOrkoAc5njBa80RcNJmiqQoKt9kMtDmitb9eEbR_mtJOYJOOL3Yw9YQ7LVAyhbvuBvUWOvgzqwgAIXXXHzErlXpJMdzS2IPlYyAu6KV8zB8rIt-zRUYpzN5m83wjVE3BfdCVXqF78OQA.8VgwM-mC-WFiCvsn.Gg3ZHge_zKuhD_9BOklUSJY2DFmMcWxHIQQKYfN_NxmC64fLaw6_VT944np3rHTfV_nGrnvmwr28le5W-3qkSO9DPbmm6icptcC2e13bSYd7WRoeOT5xOdCITgyyNvoWJqtwfJv581YfNR8E5hU25T-nBqMv0mQen_h-3I0Crz0rIHT2tRt1NlDEelpSpSXUETrL20hjO-ulZl4u-hxHG_QFG3HO3nnFt124E9nvmIQLtB0mkREFzabmZVl4UkUs_c1qcpRSoab9X_NjVolxz9iypTFMuW9l9bDhG4jfC9pmrP-Gl95adco566OqgTiqoX46MPizQQ8co6TQcgCz8I4_2wKuaEi3iVhwmZxOfwI40Abgo9QcdKHwRjCDktW-J063TJg6UYVsiw0F1Da2XBexoHLZrFQ2Xy4dh90bn2PnASu7R49NbkFIH3_MEy8QitF9CfTQ-MWXMZUXcc9aPxR3dWi3G4eb-NJfeBH_SHkc2oVz2CrLgqgWHCCDngRMcn--d4nMewNX1gRLuw8thuP37-bzC566Udfa3h81vwbMDtLgO1iedR0fjzWQnJsc

In [72]:
refresh_response = cognito_refresh_token(IdToken, refresh_token)

{'sub': 'ee0bedcf-3ad8-442a-8aaa-015570dc957e', 'email_verified': True, 'iss': 'https://cognito-idp.us-east-1.amazonaws.com/us-east-1_XEu4cVvtK', 'cognito:username': 'ee0bedcf-3ad8-442a-8aaa-015570dc957e', 'aud': '3v0e377t1vt69jcrqeo588hlh5', 'event_id': '322977c9-e1a3-4ff9-ab61-97ea80dd9504', 'token_use': 'id', 'auth_time': 1583977143, 'name': 'Aunt Bea', 'exp': 1583980743, 'iat': 1583977143, 'custom:favorite_band': 'Lawerence Welk', 'email': 'aunt.bea@contbay.com'}
ee0bedcf-3ad8-442a-8aaa-015570dc957e


In [73]:
refresh_response

{'ChallengeParameters': {},
 'AuthenticationResult': {'AccessToken': 'eyJraWQiOiJvc3I1WWozeE9Ta3h6SHg0UDl1RFQ3a1lxMFQ5Mk9VcnVFa2ltYW5BSGVjPSIsImFsZyI6IlJTMjU2In0.eyJzdWIiOiJlZTBiZWRjZi0zYWQ4LTQ0MmEtOGFhYS0wMTU1NzBkYzk1N2UiLCJldmVudF9pZCI6IjMyMjk3N2M5LWUxYTMtNGZmOS1hYjYxLTk3ZWE4MGRkOTUwNCIsInRva2VuX3VzZSI6ImFjY2VzcyIsInNjb3BlIjoiYXdzLmNvZ25pdG8uc2lnbmluLnVzZXIuYWRtaW4iLCJhdXRoX3RpbWUiOjE1ODM5NzcxNDMsImlzcyI6Imh0dHBzOlwvXC9jb2duaXRvLWlkcC51cy1lYXN0LTEuYW1hem9uYXdzLmNvbVwvdXMtZWFzdC0xX1hFdTRjVnZ0SyIsImV4cCI6MTU4Mzk4MDg1MiwiaWF0IjoxNTgzOTc3MjUyLCJqdGkiOiJmMTVlZjNlNS04MTVhLTRlOWUtYWVhZi1kMGEwMzUzMDU2MTgiLCJjbGllbnRfaWQiOiIzdjBlMzc3dDF2dDY5amNycWVvNTg4aGxoNSIsInVzZXJuYW1lIjoiZWUwYmVkY2YtM2FkOC00NDJhLThhYWEtMDE1NTcwZGM5NTdlIn0.mEYLLIBk2IG2x0w6ONsW56_XG3CPG6AEATLxe-dAGQBRWeHOoJwE7bMQQe3n7ANAc0UCkpCXmW9A2oPLOm-OisGWOfuw2sDMXgFhpEMEXpbbtwizQJjCKGkWOAvklUSa0OfeIu1NF30S77pUf26bpsUh7XhVFkRla8R5uYUYoeJVVkn3Djf-S4KpRbgFjU9PhFJD8OeFQiC37wUUhHjPLDXAxXW_mH1NHYuKt9tBlkdFlgRXI8-Fb1RcxAiMnusomVuNa45oWVfZ_w

## POST request to API Gateway

In [74]:
api_gateway_url = os.getenv('AWS_API_GATEWAY_URL')

In [75]:
response = requests.post(api_gateway_url, headers={"token":"bad token"}, data={"foo":"bar"})

In [76]:
# This should result in a message like:
# {"message":"Unauthorized"}
response.content

b'{"message":"Unauthorized"}'

In [77]:
response = requests.post(api_gateway_url, headers={"token":IdToken}, json={"note":"buy milk on way home"})

In [78]:
response

<Response [200]>

In [79]:
response.content

b'"Add Note for email: aunt.bea@contbay.com. Success"'